<a href="https://colab.research.google.com/github/fredux/DevOps/blob/main/Solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://security.ubuntu.com/ub

In [ ]:

!pip install pyspark

     |████████████████████████████████| 204.2MB 61kB/s 
     |████████████████████████████████| 204kB 45.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=133a046c3efa9b07e7c3cb564b0059196ef4876ede47c05e4dfe6dc6960fb3bc
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
!git clone https://github.com/leonardoamorim/aulapython.git

Cloning into 'aulapython'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 51 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [ ]:
! gunzip aulapython/NASA_access_log_Jul95.gz
! gunzip aulapython/NASA_access_log_Aug95.gz

In [ ]:
from pyspark import SparkConf, SparkContext
from operator import add

In [ ]:
configuracao = (SparkConf().setMaster("local").setAppName("Exercicio")
                .set("spark.executor.memory", "5g"))

In [ ]:
sc = SparkContext(conf = configuracao)

In [ ]:
julho = sc.textFile('aulapython/NASA_access_log_Jul95')
agosto = sc.textFile('aulapython/NASA_access_log_Aug95')

In [ ]:
julho = julho.cache()
agosto = agosto.cache()

In [ ]:
# Numero de hosts distintos no mes de Julho
contagem_julho = julho.map(lambda line: line.split(' ')[0]).distinct().count()
print("Numero de hosts distintos no mes de Julho:", contagem_julho)

Numero de hosts distintos no mes de Julho: 81983


In [ ]:
# Numero de hosts distintos no mes de Agosto
contagem_agosto = agosto.map(lambda line: line.split(' ')[0]).distinct().count()
print("Numero de hosts distintos no mes de Agosto:", contagem_agosto)

Numero de hosts distintos no mes de Agosto: 75060


In [ ]:
# Funcao para verificar se em uma linha existe um codigohttp igual a 404
def codigo404(linha):
    try:
        codigohttp = linha.split(' ')[-2]
        if codigohttp == '404':
            return True
    except:
        pass
    return False

In [ ]:
erros404_julho = julho.filter(codigo404).cache()
erros404_agosto = agosto.filter(lambda linha: linha.split(' ')[-2] == '404').cache()

In [ ]:
print('Erros 404 em Julho: %s' % erros404_julho.count())

Erros 404 em Julho: 10845


In [ ]:
print('Erros 404 em Agosto: %s' % erros404_agosto.count())

Erros 404 em Agosto: 10056


In [ ]:
# Funcao para retornar os top5 urls com mais erros 404
def top5_hosts404(rdd):
    hosts = rdd.map(lambda linha: linha.split('"')[1].split(' ')[1])
    counts = hosts.map(lambda hosts: (hosts, 1)).reduceByKey(add)
    top5 = counts.sortBy(lambda par: -par[1]).take(5)
    
    print('\nTop 5 urls:')
    for hosts, count in top5:
        print(hosts, count)
        
    return top5

In [ ]:
top5_hosts404(erros404_julho)


Top 5 urls:
/pub/winvn/readme.txt 667
/pub/winvn/release.txt 547
/history/apollo/apollo-13.html 286
/shuttle/resources/orbiters/atlantis.gif 232
/history/apollo/a-001/a-001-patch-small.gif 230


[('/pub/winvn/readme.txt', 667),
 ('/pub/winvn/release.txt', 547),
 ('/history/apollo/apollo-13.html', 286),
 ('/shuttle/resources/orbiters/atlantis.gif', 232),
 ('/history/apollo/a-001/a-001-patch-small.gif', 230)]

In [ ]:
top5_hosts404(erros404_agosto)


Top 5 urls:
/pub/winvn/readme.txt 1337
/pub/winvn/release.txt 1185
/shuttle/missions/STS-69/mission-STS-69.html 683
/images/nasa-logo.gif 319
/shuttle/missions/sts-68/ksc-upclose.gif 253


[('/pub/winvn/readme.txt', 1337),
 ('/pub/winvn/release.txt', 1185),
 ('/shuttle/missions/STS-69/mission-STS-69.html', 683),
 ('/images/nasa-logo.gif', 319),
 ('/shuttle/missions/sts-68/ksc-upclose.gif', 253)]

In [ ]:
# Quantidade de erros 404 por dia
def contador_dias_404(rdd):
    dias = rdd.map(lambda linha: linha.split('[')[1].split(':')[0])
    counts = dias.map(lambda dia: (dia, 1)).reduceByKey(add).collect()
    return counts

In [ ]:
contador_dias_404(erros404_julho)

[('13/Jul/1995', 532),
 ('21/Jul/1995', 334),
 ('25/Jul/1995', 461),
 ('09/Jul/1995', 348),
 ('15/Jul/1995', 254),
 ('16/Jul/1995', 257),
 ('18/Jul/1995', 465),
 ('17/Jul/1995', 406),
 ('07/Jul/1995', 570),
 ('12/Jul/1995', 471),
 ('19/Jul/1995', 639),
 ('22/Jul/1995', 192),
 ('23/Jul/1995', 233),
 ('03/Jul/1995', 474),
 ('05/Jul/1995', 497),
 ('10/Jul/1995', 398),
 ('14/Jul/1995', 413),
 ('01/Jul/1995', 316),
 ('02/Jul/1995', 291),
 ('04/Jul/1995', 359),
 ('06/Jul/1995', 640),
 ('08/Jul/1995', 302),
 ('11/Jul/1995', 471),
 ('20/Jul/1995', 428),
 ('24/Jul/1995', 328),
 ('26/Jul/1995', 336),
 ('27/Jul/1995', 336),
 ('28/Jul/1995', 94)]

In [ ]:
contador_dias_404(erros404_agosto)

[('01/Aug/1995', 243),
 ('07/Aug/1995', 537),
 ('09/Aug/1995', 279),
 ('10/Aug/1995', 315),
 ('21/Aug/1995', 305),
 ('27/Aug/1995', 370),
 ('30/Aug/1995', 571),
 ('03/Aug/1995', 304),
 ('06/Aug/1995', 373),
 ('08/Aug/1995', 391),
 ('16/Aug/1995', 259),
 ('20/Aug/1995', 312),
 ('05/Aug/1995', 236),
 ('11/Aug/1995', 263),
 ('12/Aug/1995', 196),
 ('13/Aug/1995', 216),
 ('15/Aug/1995', 327),
 ('17/Aug/1995', 271),
 ('22/Aug/1995', 288),
 ('23/Aug/1995', 345),
 ('24/Aug/1995', 420),
 ('26/Aug/1995', 366),
 ('28/Aug/1995', 410),
 ('14/Aug/1995', 287),
 ('18/Aug/1995', 256),
 ('19/Aug/1995', 209),
 ('31/Aug/1995', 526),
 ('04/Aug/1995', 346),
 ('25/Aug/1995', 415),
 ('29/Aug/1995', 420)]

In [ ]:
# Quantidade de bytes acumulados 
def quantidade_bytes_acumulados(rdd):
    def contador(linha):
        try:
            count = int(linha.split(" ")[-1])
            if count < 0:
                raise ValueError()
            return count
        except:
            return 0
        
    count = rdd.map(contador).reduce(add)
    return count

In [ ]:
print('Quantidade de bytes total em Julho: %s' % quantidade_bytes_acumulados(julho))

Quantidade de bytes total em Julho: 38695973491


In [ ]:
print('Quantidade de bytes total em Agosto: %s' % quantidade_bytes_acumulados(agosto))

Quantidade de bytes total em Agosto: 26828341424


In [ ]:
sc.stop()